In [ ]:
#hide
from memery.core import indexFlow, queryFlow
from ipyplot import plot_images

/home/mage/.local/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


# memery

> Search over large image datasets with computer vision!

## Install

`pip install memery`

## How to use

Simply use queryFlow to search over a folder recursively!

In [ ]:
ranked = queryFlow('./images', 'a funny dog meme')

Searching 78 images


In [ ]:
plot_images(ranked, max_images=6)

## TODO:

- Documentation and tests in notebooks
- Interactive GUI
- Optimize the image loader and number of trees based on memory and db size
- Type annotations

## DONE:
- _Code for joining archived data to new data_
- _Code for saving indexes to archive_
- _Flows_
- _Cleanup repo_
- _CLI_
